# SFC, FC & SC - LOBE vs HCP #

In [2]:
subjects=snakemake.params.subjects

#for dataset in subjects.keys():
 #   subjects[dataset] = [ 'sub-'+subject for subject in subjects[dataset] ]

NameError: name 'snakemake' is not defined

In [5]:
subjects

NameError: name 'subjects' is not defined

## Importing packages and setting paths ##

In [15]:
import pandas as pd
import nibabel as nib
from glob import glob
import numpy as np


In [16]:
#print(f"snakemake.params.subjects[{dataset}] = {snakemake.params.subjects[dataset]}")

In [17]:
#first load up the label dseg table
atlas='brainnetome'

df_labels = pd.read_csv(f'resources/atlas/atlas-{atlas}_dseg.tsv',sep='\t')
df_labels

df_network_names = df_labels.networks.unique()
df_network_names

df_tabular = pd.read_csv(snakemake.input.tsv,sep='\t')
#df_tabular = pd.read_csv('results/analysis/df_tabular.csv',sep='\t')

df_tabular



FileNotFoundError: [Errno 2] No such file or directory: 'resources/atlas/atlas-brainnetome_dseg.tsv'

In [26]:
# Schaefer Patterns
#fc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_bold.pconn.nii'
#sc_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-91k_atlas-schaefer_struc.pconn.nii'
#sfc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii'

# Brainnetome Patterns
# Schaefer Patterns
fc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-brainnetome_bold.pconn.nii'
sc_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-91k_atlas-brainnetome_struc.pconn.nii'
sfc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-brainnetome_sfc.pscalar.nii'




In [24]:
sfc_paths={ dataset: sorted([sfc_pattern.format(dataset=dataset,subject=subject) for subject in subjects[dataset]]) 
               for dataset in subjects.keys() }

fc_paths={ dataset: sorted([fc_pattern.format(dataset=dataset,subject=subject) for subject in subjects[dataset]]) 
               for dataset in subjects.keys() }

sc_paths={ dataset: sorted([sc_pattern.format(dataset=dataset,subject=subject) for subject in subjects[dataset]]) 
               for dataset in subjects.keys() }

In [25]:
#from snakebids.utils.snakemake_io import glob_wildcards
#subjects = glob_wildcards(sfc_pattern.format(dataset='HCP',subject='{subject}'))

if 'LOBE' in sfc_paths:
    print(f"Number of files in sfc_paths['LOBE']: {len(sfc_paths['LOBE'])}")
else:
    print("'LOBE' key is not in sfc_paths.")


Number of files in sfc_paths['LOBE']: 0


In [23]:
sfc={}
fc={}
sc={}
num_parcels = nib.load(sfc_paths['LOBE'][0]).get_fdata().shape[1]
datasets=['HCP','LOBE']
for dataset in datasets:
    sfc[dataset] = np.concatenate(  [ nib.load(path).get_fdata()  for path in sfc_paths[dataset]] ,axis=0)
    fc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in fc_paths[dataset]] ,axis=0)
    sc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sc_paths[dataset]] ,axis=0)

IndexError: list index out of range

In [24]:
print(df_tabular)

     participant_id dataset        age sex  age_seizure_onset  \
0    sub-HCD0001305     HCP  11.916667   M                NaN   
1    sub-HCD0021614     HCP   9.166667   F                NaN   
2    sub-HCD0026119     HCP  15.166667   F                NaN   
3    sub-HCD0041822     HCP  17.416667   M                NaN   
4    sub-HCD0042420     HCP  18.333333   M                NaN   
..              ...     ...        ...  ..                ...   
357         sub-029    LOBE  16.000000   M               15.0   
358         sub-031    LOBE   9.000000   M                4.0   
359         sub-033    LOBE   7.000000   M                4.0   
360         sub-035    LOBE   7.000000   M                2.0   
361         sub-038    LOBE  10.000000   F                7.0   

     age_ep_diagnosis  ep_duration_first_scan  seizure_duration_first_scan  \
0                 NaN                     NaN                          NaN   
1                 NaN                     NaN                  

In [25]:

#this adds RSFC features as new columns to the df_tabular

df_list=[]
for dataset in datasets:
    df_=pd.DataFrame(sfc[dataset],
                 columns=('RSFC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    #df_['participant_id'] = snakemake.params.subjects[dataset]

    # Check that the number of rows in df_ matches the number of subjects
    participant_ids = snakemake.params.subjects[dataset]
    
    # Assign participant IDs directly to df_
    df_['participant_id'] = participant_ids
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

NameError: name 'snakemake' is not defined

In [ ]:
print(f"Length of sfc[dataset] for 'LOBE': {len(sfc[dataset])}")


## Average FC per parcel ##

In [26]:
#set diagonal to be nan
x1 = fc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = fc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [27]:
x1.shape

(78, 246, 246)

In [28]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

/tmp/ipykernel_1428816/3377732870.py:2: RuntimeWarning: Mean of empty slice
  x2 = np.nanmean(x2,axis=2)


In [29]:
x1.shape

(78, 246)

In [30]:
x2.shape

(27, 246)

In [ ]:
#add FC parcel avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('RFC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    df_['participant_id'] = snakemake.params.subjects[dataset]

    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

# Structural Connectivity #

## Average SC per parcel ##

In [31]:
#set diagonal to be nan
x1 = sc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [32]:
x1.shape

(77, 246, 246)

In [33]:
x1 = np.nanmean(x1,axis=2)
x1.shape
x2 = np.nanmean(x2,axis=2)
x2.shape

(27, 246)

In [34]:
x1.shape

(77, 246)

In [35]:
x2.shape

(27, 246)

In [3]:
#add SC parcel avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('RSC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    df_['participant_id'] = snakemake.params.subjects[dataset]

    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

NameError: name 'x1' is not defined

In [ ]:
df_tabular

# Network-Wise, LOBE vs HCP #

## Paths ##

In [37]:
# Brainnetome Patterns
fc_network_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-brainnetome_netbold.pconn.nii'
sc_network_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-91k_atlas-brainnetome_netstruc.pconn.nii'
sfc_network_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-91k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-brainnetome_netsfc.pconn.nii'


In [38]:
sfc_netpaths={}
sfc_netpaths['HCP'] = sorted(glob(sfc_network_pattern.format(dataset='HCP',subject='*')))
sfc_netpaths['LOBE'] = sorted(glob(sfc_network_pattern.format(dataset='LOBE',subject='*')))

fc_netpaths={}
fc_netpaths['HCP'] = sorted(glob(fc_network_pattern.format(dataset='HCP',subject='*')))
fc_netpaths['LOBE'] = sorted(glob(fc_network_pattern.format(dataset='LOBE',subject='*')))

sc_netpaths={}
sc_netpaths['HCP'] = sorted(glob(sc_network_pattern.format(dataset='HCP',subject='*')))
sc_netpaths['LOBE'] = sorted(glob(sc_network_pattern.format(dataset='LOBE',subject='*')))

#print(sfc_netpaths)
#print(fc_netpaths)
#print(sc_netpaths)

In [40]:
sfc={}
fc={}
sc={}
num_parcels = nib.load(sfc_netpaths[dataset][0]).get_fdata().shape[1]

for dataset in datasets:

    sfc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sfc_netpaths[dataset]] ,axis=0)  
    #for path in sfc_netpaths[dataset]] ,axis=0)
    fc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in fc_netpaths[dataset]] ,axis=0)
    sc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sc_netpaths[dataset]] ,axis=0)

# Network SFC #

In [144]:
data_={}
for dataset in ['HCP','LOBE']:

    x1 = sfc[dataset]
    data_[dataset] = np.zeros((x1.shape[0],int((num_parcels*(num_parcels+1))/2)))
    num_parcels = x1.shape[1]  # Assuming it's 8

    # Loop over each "slice" in x2 (assuming only one slice, so loop over the first element)
    for s in range(x1.shape[0]):
        conn = x1[s, :, :]  # Extract the 8x8 matrix for the current subject        
        # Now loop over the upper triangle to gather values for each "mixture"
        values_subj=[]
        for i in range(num_parcels):
            for j in range(i, num_parcels): # Include diagonal as well by starting at i
               values_subj.append(conn[i, j])  # Get the value for that specific pair (i, j)
    
        data_[dataset][s,:] = values_subj

## Network SFC to DF ##

In [ ]:
df_list = []

for dataset in datasets:
    colnames=[]
    for i in range(len(df_network_names)):
        for j in range(i, len(df_network_names)):

            colnames.append(f'NSFC_{df_network_names[i]}_{df_network_names[j]}')
                

    df_ = pd.DataFrame(data_[dataset], columns=colnames)
    
    # Filter the main df_tabular by the current dataset
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    
    # Add the participant_ids from the subjects
    #df_['participant_id'] = snakemake.params.subjects[dataset]
    df_['participant_id'] = ['sub-'+subject for subject in subjects[dataset]]
    
    # Merge the new df_ with df_tabular_ on 'participant_id'
    df_merged_ = pd.merge(df_tabular_, df_, how='left', on='participant_id')
    
    # Append merged DataFrame to the list
    df_list.append(df_merged_)

# Concatenate all merged DataFrames
df_tabular = pd.concat(df_list)


In [146]:
# # delete if above works
# data_={'HCP':x1,'LOBE':x2}  

# # Example datasets
# datasets = ['LOBE', 'HCP']

# # Example: x2 is a (1, 8, 8) matrix
# #changed
# #x2 = x2[0]  # Now x2 is shape (8, 8)

# # Assuming df_tabular is already loaded
# # df_tabular = pd.read_csv("your_file.csv")  # Or however you load df_tabular

# # List to collect merged DataFrames
# df_list = []

# for dataset in datasets:
#     # Create column names: NSFC_network1_network2
#     colnames=[]
#     for i in range(len(df_network_names)):
#         for j in range(i, len(df_network_names)):
#             if i>j:
#                 continue
#             colnames.append(f'NSFC_{df_network_names[i]}_{df_network_names[j]}')
                
       

#     # Flatten the upper triangle of x2 into a 1D array to match the number of columns
#     upper_triangle_values = [x2[i, j] for i in range(len(df_network_names)) for j in range(i, len(df_network_names))]

#     # Create a DataFrame for the current dataset using the flattened upper triangle values
#     df_ = pd.DataFrame([upper_triangle_values], columns=colnames)
    
#     # Filter the main df_tabular by the current dataset
#     df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    
#     # Add the participant_ids from the subjects
#     df_['participant_id'] = snakemake.params.subjects[dataset]
#     #df_['participant_id'] = subjects[dataset]
    
#     # Merge the new df_ with df_tabular_ on 'participant_id'
#     df_merged_ = pd.merge(df_tabular_, df_, how='left', on='participant_id')
    
#     # Append merged DataFrame to the list
#     df_list.append(df_merged_)

# # Concatenate all merged DataFrames
# df_tabular_final = pd.concat(df_list)

# # Display the final DataFrame
# df_tabular_final


NameError: name 'snakemake' is not defined

# Network FC #

In [ ]:
data_={}
for dataset in ['HCP','LOBE']:

    x1 = fc[dataset]
    data_[dataset] = np.zeros((x1.shape[0],int((num_parcels*(num_parcels+1))/2)))
    num_parcels = x1.shape[1]  # Assuming it's 8

    for s in range(x1.shape[0]):
        conn = x1[s, :, :]  # Extract the 8x8 matrix for the current subject        
        # Now loop over the upper triangle to gather values for each "mixture"
        values_subj=[]
        for i in range(num_parcels):
            for j in range(i, num_parcels): # Include diagonal as well by starting at i

               values_subj.append(conn[i, j])  # Get the value for that specific pair (i, j)
    
        data_[dataset][s,:] = values_subj

## Network FC to DF ##

In [ ]:
df_list = []

for dataset in datasets:
    colnames=[]
    for i in range(len(df_network_names)):
        for j in range(i, len(df_network_names)):

            colnames.append(f'NFC_{df_network_names[i]}_{df_network_names[j]}')
                
    df_ = pd.DataFrame(data_[dataset], columns=colnames)
    
    # Filter the main df_tabular by the current dataset
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    
    # Add the participant_ids from the subjects
    #df_['participant_id'] = snakemake.params.subjects[dataset]
    df_['participant_id'] = ['sub-'+subject for subject in subjects[dataset]]
    
    # Merge the new df_ with df_tabular_ on 'participant_id'
    df_merged_ = pd.merge(df_tabular_, df_, how='left', on='participant_id')
    
    # Append merged DataFrame to the list
    df_list.append(df_merged_)

# Concatenate all merged DataFrames
df_tabular = pd.concat(df_list)

# Network SC #

In [ ]:
data_={}
for dataset in ['HCP','LOBE']:

    x1 = sc[dataset]
    data_[dataset] = np.zeros((x1.shape[0],int((num_parcels*(num_parcels+1))/2)))
    num_parcels = x1.shape[1]  # Assuming it's 8

    # Loop over each "slice" in x2 (assuming only one slice, so loop over the first element)
    for s in range(x1.shape[0]):
        conn = x1[s, :, :]  # Extract the 8x8 matrix for the current subject        
        # Now loop over the upper triangle to gather values for each "mixture"
        values_subj=[]
        for i in range(num_parcels):
            for j in range(i, num_parcels): # Include diagonal as well by starting at i

               values_subj.append(conn[i, j])  # Get the value for that specific pair (i, j)
    
        data_[dataset][s,:] = values_subj

## Network SC to DF ##

In [ ]:
df_list = []

for dataset in datasets:
    colnames=[]
    for i in range(len(df_network_names)):
        for j in range(i, len(df_network_names)):

            colnames.append(f'NSC_{df_network_names[i]}_{df_network_names[j]}')

    df_ = pd.DataFrame(data_[dataset], columns=colnames)
    
    # Filter the main df_tabular by the current dataset
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    
    # Add the participant_ids from the subjects
    #df_['participant_id'] = snakemake.params.subjects[dataset]
    df_['participant_id'] = ['sub-'+subject for subject in subjects[dataset]]
    
    # Merge the new df_ with df_tabular_ on 'participant_id'
    df_merged_ = pd.merge(df_tabular_, df_, how='left', on='participant_id')
    
    # Append merged DataFrame to the list
    df_list.append(df_merged_)

# Concatenate all merged DataFrames
df_tabular = pd.concat(df_list)

## Save regional and network SFC, SC & FC to masterlist ##

In [ ]:
df_tabular.to_csv(snakemake.output.tsv,sep='\t',index=False)


df_tabular 